### 7.被管理的属性

#### 7.1 property

property 协议允许我们把一个特定属性的获取/赋值/删除等操作指向我们所提供的函数。通过 property 可以实现懒加载的动态属性，只有在访问时才计算并返回一个值。

用法：

1. attribute = property(fget, fset, fdel, doc) # 没有fset就变成了只读的
2. 装饰器

In [1]:
class Person:
    
    def __init__(self, name): self._name = name
        
    def get_name(self):
        print('getting name')
        return self._name
    
    def set_name(self, val):
        print('setting name')
        self._name = val
        
    def del_name(self):
        print('deleting name')
        del self._name
        
    name = property(get_name, set_name, del_name, 'name property docs')
    
jade = Person('jade')
jade.name

In [2]:
jade.name = 'Jade'

In [3]:
del jade.name

In [4]:
class Person:

    def __init__(self, name): self._name = name

    @property
    def name(self):
        print('fetching name')
        return self._name
    
    @name.setter
    def name(self, val):
        print('changing name')
        self._name = val
    
    @name.deleter
    def name(self):
        print('removing name')
        del self._name
        
bob = Person('bob')
bob.name

'bob'

In [5]:
bob.name = 'Bob'

changing name


In [6]:
del bob.name

removing name


#### 7.2 描述器

descriptor 提供了拦截属性访问的另一种方法，它允许我们将一个特定属性的操作指向我们提供的一个单独的类对象的方法。即，一个描述器只能管理单个的，指定的属性。

property 实际上就是一个描述器创建的简便方式。

所有带有：

1. `__get__(self, instance, owner)`
2. `__set__(self, instance, value)`
3. `__delete__(self, instance)`

方法的类都可以称之为描述器。其中，owner 指的是这个描述器实例所依附的类，而 instance 要么是被访问的属性所属的实例，要么所访问的属性直接属于类时是None。

In [7]:
class IntDescriptor:
    
    number = 0
    
    def __get__(self, instance, owner):
        print('getting, args: ', self, instance, owner)
        return self.number
    
    # 没有此方法也不会变成只读的，需要通过引发异常来支持只读属性
    def __set__(self, instance, value):
        print('setting')
        if not isinstance(value, int): raise ValueError('this property should be an integer')
        self.number = value
        
    def __delete__(self, instance):
        print('deleting')
        del self
        
class Number:
    
    val = IntDescriptor()
    
num = Number()
num.val

getting, args:  <__main__.IntDescriptor object at 0x10777c150> <__main__.Number object at 0x10777c2d0> <class '__main__.Number'>


0

In [8]:
num.val = 10

setting


In [9]:
del num.val

deleting


In [10]:
Number.val

getting, args:  <__main__.IntDescriptor object at 0x10777c150> None <class '__main__.Number'>


10

### 8. 装饰器

#### 8.1 函数装饰器
装饰器自身是一个返回可调用对象的可调用对象。本质上是进行函数名称的重绑定。其中嵌套函数编写的函数装饰器是最佳实践。

In [11]:
def decorator(func):
    def wrapper(*args):
        print('actually calling wrapper')
        return func(*args)
    return wrapper

# func = decorator(func)
@decorator
def func(x, y):
    return x + y

func(1, 2)

actually calling wrapper


3

#### 8.2 类装饰器
类装饰器不包装单个函数或方法而是进行类的管理或是使用额外逻辑来实现实例构造应用的一种方式。

In [12]:
def decorator(cls):
    class Wrapper:
        def __init__(self, *args):
            self.wrapped = cls(*args)
            
        def __getattr__(self, name):
            return getattr(self.wrapped, name)
    return Wrapper

@decorator
class C:
    def __init__(self, x, y):
        self.attr = 'spam'

c = C(1, 2)
type(c)

__main__.decorator.<locals>.Wrapper

In [13]:
def singleton(cls):
    instance = None
    def wrapper(*args, **kargs):
        nonlocal instance
        if not instance:
            instance = cls(*args, **kargs)
        return instance
    return wrapper

@singleton
class S:
    
    def __init__(self, name):
        self.name = name

s1 = S('s')
s2 = S('s')
s1 is s2

True

In [14]:
def dec(A, B):
    print(A, B)
    def actuall(func):
        def call(*args):
            return func(*args)
        return call
    return actuall

@dec(A='hello', B='param')
def func(x, y):
    return x + y

func(1, 2)

hello param


3

### 9. 元类

在 Python 中类是 type 的实例，实例创建自类，而类创建自 type。元类是 type 的子类，通过 class 协议来拦截类的创建和初始化过程。

class 的过程实际上是：
```python
class = type(classname, superclasses, attributedict)
```

type 对象定义了一个 `__call__` 运算符重载方法，当被调用时，该方法运行：

1. type.__new__(typeclass, classname, superclasses, attributedict)
2. type.__init__(class, classname, superclasses, attributedict)

In [15]:
class Meta(type):
    def __new__(meta, classname, supers, classdict):
        print(meta, classname, supers, classdict)
        return type.__new__(meta, classname, supers, classdict)

class Eggs: pass

class Spam(Eggs, metaclass=Meta):
    data = 1
    def meth(self, arg):
        return self.data + arg
    
X = Spam()

<class '__main__.Meta'> Spam (<class '__main__.Eggs'>,) {'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x10778c950>}


In [16]:
X.meth(1)

2

In [17]:
class MetaObj:
    '''
    通过普通类模仿 type 机制
    '''
    
    def __call__(self, classname, supers, classdict):
        print('In MetaObj.call:', classname, supers, classdict, sep='\n')
        Class = self.__New__(classname, supers, classdict)
        self.__Init__(Class, classname, supers, classdict)
        return Class
    
    def __New__(self, classname, supers, classdict):
        print('In MetaObj.new: ', classname, supers, classdict, sep='\n')
        return type(classname, supers, classdict)
    
    def __Init__(self, Class, classname, supers, classdict, sep='\n'):
        print('In MetaObj.init: ', classname, supers, classdict, sep='\n')
        
class Super: pass
class Sub(Super, metaclass=MetaObj()):
    data = 1
    def add(self, arg):
        return self.data + arg

sub = Sub()
sub.add(2)

In MetaObj.call:
Sub
(<class '__main__.Super'>,)
{'__module__': '__main__', '__qualname__': 'Sub', 'data': 1, 'add': <function Sub.add at 0x10779b560>}
In MetaObj.new: 
Sub
(<class '__main__.Super'>,)
{'__module__': '__main__', '__qualname__': 'Sub', 'data': 1, 'add': <function Sub.add at 0x10779b560>}
In MetaObj.init: 
Sub
(<class '__main__.Super'>,)
{'__module__': '__main__', '__qualname__': 'Sub', 'data': 1, 'add': <function Sub.add at 0x10779b560>}


3

In [18]:
def square(obj):
    return obj.val ** 2

class Extender(type):
    
    def __new__(meta, classname, supers, classdict):
        print('adding func square')
        classdict['square'] = square
        return type.__new__(meta, classname, supers, classdict)
    
class Operator(metaclass=Extender):
    
    def __init__(self, val):
        self.val = val
        
op = Operator(9)
op.square()

adding func square


81